In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from tqdm import tqdm
from collections import defaultdict
import cv2 as cv

## Random Maps

In [ ]:
def load_map(path):
    with open(path, "r") as f:
        lines = f.readlines()
    height = int(lines[1].split()[1])
    width = int(lines[2].split()[1])
    cells = np.zeros((height, width))
    for i, line in enumerate(lines[4:]):
        for j, char in enumerate(line.strip()):
            if char == '@':
                cells[i, j] = 1
            elif char == '.' or char == "T":
                cells[i, j] = 0
            else:
                ValueError('Incorrect cell type')
    return cells

In [ ]:
load_dir = Path("/archive/savkin/raw_datasets/MIPT/TransPath_data/maps")

map = load_map(load_dir / "random-map" / "random512-40-9.map")

plt.imshow(map)
plt.show()

In [ ]:
random_maps_dir = load_dir / "random-map"
map_densities = np.arange(10, 41, 5)
random_maps = []
for density in tqdm(map_densities):
    for i in range(10):
        map = load_map(random_maps_dir / f"random512-{density}-{i}.map")
        random_maps.append(map)

random_maps = np.array(random_maps)
random_maps.shape

In [ ]:
np.save(random_maps_dir / "all_rand_maps.npy", random_maps)

## Street Maps

In [ ]:
street_maps_dir = load_dir / "street-map"
street_maps = defaultdict(list)
for path in tqdm(street_maps_dir.iterdir()):
    map = load_map(path)
    edge_size = map.shape[0]
    street_maps[edge_size].append(map)

In [ ]:
for edge_size, maps in street_maps.items():
    np.save(load_dir / "my_maps" / f"street_maps_{edge_size}.npy", maps)

## Tile MP Dataset

In [ ]:
path = "/home/logiczmaksimka/Documents/MIPT/10sem/heuristics/TransPath-Project/data/all_128x2.npz"

data = np.load(path)

Let's take a look at generated cf

In [ ]:
cf_pred = np.load("/home/logiczmaksimka/Downloads/val/cf_pred.npy")
cf_true = np.load("/home/logiczmaksimka/Downloads/val/cf.npy")

i = 0
p = cf_pred[i][0]
t = cf_true[i][0]


plt.imshow(p, vmin=0, vmax=1)
plt.show()

plt.imshow(t)
plt.show()

## Augmentations

In [2]:
load_dir = Path("/home/savkin/MIPT/2024/TransPath Project/data")
load_path = load_dir / "all_32x8.npz"
maps = np.load(load_path)
maps.keys()

KeysView(<numpy.lib.npyio.NpzFile object at 0x7fa9006e9df0>)

In [3]:
def augment_map(map):
    rotation_types = [cv.ROTATE_90_CLOCKWISE, cv.ROTATE_180, cv.ROTATE_90_COUNTERCLOCKWISE]
    fliped_map = cv.flip(map, 0)
    augmented_maps = [
        map,
        *[cv.rotate(map, rotation_type) for rotation_type in rotation_types],
        fliped_map,
        *[cv.rotate(fliped_map, rotation_type) for rotation_type in rotation_types]
    ]
    return augmented_maps

In [5]:
augmented_maps = {}
for orig_name, new_name in [("arr_0", "train"), ("arr_1", "valid"), ("arr_2", "test")]:
    generated_maps = []
    for map in tqdm(maps[orig_name]):
        generated_maps = generated_maps + augment_map(map)
    augmented_maps[new_name] = generated_maps

# quick check for duplicates
x = [map for map in augmented_maps["train"][:8]]
for i in range(len(x)):
    for j in range(len(x)):
        print(np.prod(x[i] == x[j]), " ", end="")
    print()

100%|██████████| 400/400 [00:00<00:00, 428.23it/s]

1  0  0  0  0  0  0  0  
0  1  0  0  0  0  0  0  
0  0  1  0  0  0  0  0  
0  0  0  1  0  0  0  0  
0  0  0  0  1  0  0  0  
0  0  0  0  0  1  0  0  
0  0  0  0  0  0  1  0  
0  0  0  0  0  0  0  1  


# Generating goal nodes

In [6]:
def suggest_goals(map, n=10):
    empty_cells = np.array(list(zip(*np.where(map == 1))))
    goal_ids = np.random.choice(np.arange(len(empty_cells)), size=n, replace=False)
    goal_cells = empty_cells[goal_ids]

    # goal_maps = np.full((n, *map.shape), 0)
    # for i, cell in enumerate(goal_cells):
    #     goal_maps[i][cell[0], cell[1]] = 1

    return goal_cells

In [ ]:
map = augmented_maps["train"][1].copy()
map = map[:64, :64]
goal_maps = suggest_goals(map)
# plt.imshow(map)
# plt.show()
plt.imshow(goal_maps[0])
plt.show()

In [7]:
tasks = {}
for data_type in ["train", "valid", "test"]:
    new_tasks = []
    for map in tqdm(augmented_maps[data_type]):
        new_tasks.append(suggest_goals(map))
    tasks[data_type] = new_tasks

100%|██████████| 3200/3200 [01:18<00:00, 40.80it/s]


Saving

In [8]:
save_path = "/archive/savkin/raw_datasets/MIPT/TransPath_data/tile_mp_256/all_32x8_augmented_goals.npz"
np.savez(save_path, train=tasks["train"], valid=tasks["valid"], test=tasks["test"])

In [9]:
save_path = "/archive/savkin/raw_datasets/MIPT/TransPath_data/tile_mp_256/all_32x8_augmented_maps.npz"
np.savez(save_path, train=augmented_maps["train"], valid=augmented_maps["valid"], test=augmented_maps["test"])

In [ ]:
save_path = "/archive/savkin/raw_datasets/MIPT/TransPath_data/tile_mp_256/all_32x8_augmented.npy"
np.save(save_path, augmented_maps)

# Generating true distances

In [ ]:
from cf_generation import fill_true_dists_8_way, diagonal_distance, Map, Node, extract_node_pos

cf_pred_arr = []
# for i in tqdm(range(10)):
#     map = Map(maps[i][0])
for map in tqdm(maps):
    # cf_true = cf[i][0]

    # start = Node(*extract_node_pos(starts[i][0]))
    goal_node = Node(*extract_node_pos(goals[i][0]))
    cf_pred = fill_true_dists_8_way(goal_node, map)
    cf_pred_arr.append([cf_pred])

cf_pred_arr = np.array(cf_pred_arr)

## Generating start and goal nodes

In [ ]:
map = augmented_maps["train"][0]


Calc map hardness

In [ ]:
## обязательно проверь сложность карт!!!!!! Желательно прям добавить её к каждой карте!!!! Раз делаешь BFS то это совсем просто!!!